# Install

In [ ]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 6.6 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm.auto import tqdm
import time

import os
import random

import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import timm
from sklearn.metrics import f1_score, accuracy_score
from PIL import Image

device = torch.device('cuda')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## 각종 변수 선언

In [ ]:
batch_size = 16
epochs = 25
SEED = 35

In [ ]:
train_png = sorted(glob('/content/gdrive/MyDrive/경쟁모드/김유환/data/train512/*.png'))
test_png = sorted(glob('/content/gdrive/MyDrive/경쟁모드/김유환/data/test512/*.png'))

In [ ]:
train_y  = pd.read_csv('/content/gdrive/MyDrive/경쟁모드/정다예/project/train_df.csv')

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

## 함수선언

In [ ]:
def img_load(path):
    img = Image.open(path).convert("RGB")
    return img

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

#SEED 고정
seed_everything(SEED)

def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

## augmetation에 사용할 transforms 생성

In [ ]:
horizontal_transforms = transforms.Compose([ 
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(), 
                                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]) 

In [ ]:
vertical_transforms = transforms.Compose([ 
                                        transforms.RandomVerticalFlip(),
                                        transforms.ToTensor(), 
                                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]) 

In [ ]:
affine_transforms = transforms.Compose([ 
                                        transforms.RandomAffine((-45, 45)),
                                        transforms.ToTensor(), 
                                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]) 

In [ ]:
default_transforms = transforms.Compose([                                         
                                        transforms.ToTensor(), 
                                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]) 

In [ ]:
test_transforms = transforms.Compose([
                                        transforms.ToTensor(), 
                                        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

## Custom dataset 클래스 선언

In [ ]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train', transforms=None):
        self.img_paths = img_paths
        self.labels = labels
        self.mode = mode
        self.transforms = transforms
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        random.seed(seed)
        img = self.img_paths[idx]
        if self.transforms is not None and 'good' not in self.labels :
            img = self.transforms(img)
        else :
            img = transforms.ToTensor()(img)
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b1', pretrained=True, num_classes=88)
        
    def forward(self, x):
        x = self.model(x)
        return x

# Data load

In [ ]:
train_imgs = [img_load(m) for m in tqdm(train_png)]

  0%|          | 0/4277 [00:00<?, ?it/s]

## train dataset concat

In [ ]:
train_dataset_t1 = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train', transforms=horizontal_transforms)
train_dataset_t2 = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train', transforms=vertical_transforms)
train_dataset_t3 = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train', transforms=affine_transforms)
train_dataset_t4 = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train', transforms=default_transforms)

In [ ]:
train_sets = torch.utils.data.ConcatDataset([train_dataset_t1, train_dataset_t2, train_dataset_t3, train_dataset_t4])
train_loader = DataLoader(train_sets, shuffle=True, batch_size=batch_size, num_workers=2)

In [ ]:
import gc
del train_imgs, train_sets, train_labels, train_dataset_t1, train_dataset_t2, train_dataset_t3, train_dataset_t4

In [ ]:
gc.collect()

300

In [ ]:
torch.cuda.empty_cache()
torch.cuda.memory_reserved() 

0

# EDA

In [ ]:
train_y.head()

,index,file_name,class,state,label
0,0,10000.png,transistor,good,transistor-good
1,1,10001.png,capsule,good,capsule-good
2,2,10002.png,transistor,good,transistor-good
3,3,10003.png,wood,good,wood-good
4,4,10004.png,bottle,good,bottle-good


In [ ]:
print('class :',  train_y['class'].nunique())
print('state :', train_y['state'].nunique())
print('label : ',  train_y['label'].nunique())

class : 15
state : 49
label :  88


# Local augmentation (적용 X)

## Split images by label

In [ ]:
from PIL import Image
import PIL.ImageOps    

In [ ]:
label_list = train_y["label"].unique().tolist()
label_list[:5]

In [ ]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
 
for i in range(len(label_list)):
    createFolder(f'/content/gdrive/MyDrive/경쟁모드/정다예/train/{label_list[i]}')  

In [ ]:
train_folder = os.listdir('/content/gdrive/MyDrive/경쟁모드/정다예/train')
len(train_folder)

In [ ]:
import shutil

for i in range(len(train_png)):
    if train_png[i][-3:] == "png":  
        label = train_y.iloc[i, :]['label']
        file_source = train_png[i]
        file_destination = f'/content/gdrive/MyDrive/경쟁모드/정다예/train/{label}/' 
        shutil.copy(file_source, file_destination)

## Augmentation

In [ ]:
num_augmented_images = 30

for labels in train_folder :
    if 'good' not in labels :
        file_path = f'/content/gdrive/MyDrive/경쟁모드/정다예/train/{labels}' 
        file_names = os.listdir(file_path)
        total_origin_image_num = len(file_names)

        for i in range (0, len(file_names)) :
            origin_image_path = file_path +'/'+ file_names[i]
            image = Image.open(origin_image_path)
            for num in range (num_augmented_images) :
                random_augment = random.randrange(1,3)
                if random_augment==1:
                    #이미지 상하 반전
                    inverted_image = image.transpose(Image.FLIP_TOP_BOTTOM)
                    inverted_image.save(file_path + '_inverted_tb_' + str(num) + '.png')
                else :
                    #이미지 좌우 반전 
                    inverted_image = image.transpose(Image.FLIP_LEFT_RIGHT)
                    inverted_image.save(file_path + '_inverted_lr_' + str(num) + '.png')

In [ ]:
for i in range(len(train_png)):
        label = train_y.iloc[i, :]['label']
        file_source = train_png[i]
        file_destination = f'/content/gdrive/MyDrive/경쟁모드/정다예/train/{label}/' 
        shutil.move(file_source, file_destination)

In [ ]:
import shutil

labels = train_y["label"].unique().tolist()
for label in labels :
        file_path = f'/content/gdrive/MyDrive/경쟁모드/정다예/train/{label}/' 
        file_names = os.listdir(file_path)
        total_origin_image_num = len(file_names)
        for i in range (total_origin_image_num) : 
          file_start = f'{file_path}{file_names[i]}' 
          file_destination = f'/content/gdrive/MyDrive/경쟁모드/정다예/train512/' 
          shutil.move(file_start, file_destination)

In [ ]:
train_png = sorted(glob('/content/gdrive/MyDrive/경쟁모드/정다예/data/train512/*.png'))
train_dataset = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train')

# Train

In [ ]:
model = Network().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler() 

best=0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
    
    train_f1 = score_function(train_y, train_pred)

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b1-533bc792.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b1-533bc792.pth


epoch : 1/25    time : 381s/9143s
TRAIN    loss : 0.58117    f1 : 0.43793
epoch : 2/25    time : 361s/8299s
TRAIN    loss : 0.14014    f1 : 0.82675
epoch : 3/25    time : 360s/7922s
TRAIN    loss : 0.05457    f1 : 0.93918
epoch : 4/25    time : 359s/7547s
TRAIN    loss : 0.03197    f1 : 0.96690
epoch : 5/25    time : 359s/7174s
TRAIN    loss : 0.02263    f1 : 0.97881
epoch : 6/25    time : 360s/6836s
TRAIN    loss : 0.02331    f1 : 0.97916
epoch : 7/25    time : 359s/6464s
TRAIN    loss : 0.02035    f1 : 0.98216
epoch : 8/25    time : 359s/6104s
TRAIN    loss : 0.01518    f1 : 0.98314
epoch : 9/25    time : 358s/5729s
TRAIN    loss : 0.01483    f1 : 0.98685
epoch : 10/25    time : 358s/5366s
TRAIN    loss : 0.01295    f1 : 0.98525
epoch : 11/25    time : 357s/4999s
TRAIN    loss : 0.01287    f1 : 0.98928
epoch : 12/25    time : 358s/4655s
TRAIN    loss : 0.01347    f1 : 0.98779
epoch : 13/25    time : 358s/4297s
TRAIN    loss : 0.01113    f1 : 0.98926
epoch : 14/25    time : 358s/3934s

In [ ]:
del train_loader
gc.collect()

0

# Test data load

In [ ]:
test_imgs = [img_load(n) for n in tqdm(test_png)]

  0%|          | 0/2154 [00:00<?, ?it/s]

In [ ]:
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test', transforms=test_transforms)
del test_imgs
gc.collect()
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)
del test_dataset
gc.collect()

0

# Evaluation

In [ ]:
model.eval()
f_pred = []

with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

In [ ]:
del test_loader
gc.collect()

32

In [ ]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

# Submission

In [ ]:
submission = pd.read_csv('/content/gdrive/MyDrive/경쟁모드/정다예/project/sample_submission.csv')

submission["label"] = f_result

submission.head()
#2154 rows x 2 cols

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-good
3,3,tile-gray_stroke
4,4,tile-good


In [ ]:
submission.to_csv("/content/gdrive/MyDrive/경쟁모드/정다예/project/230316_1.csv", index = False)